# RBC Classification (Grayscale)


In [18]:
import os
import json
from pathlib import Path
from typing import List

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import visualkeras
#import cv2
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.utils import to_categorical

from PIL import Image, ImageOps, ImageFilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [33]:
# Data preparation parameters
DATASET_DIR = "bloodcells_dataset"  
IMG_SIZE = 100  # Downsize size before cropping
CROP_FACTOR = 0.6  
CROPPED_SIZE = int(IMG_SIZE * CROP_FACTOR)  # Final size after cropping (120x120 with 0.6 factor)
VAL_FRAC = 0.15
TEST_FRAC = 0.15
LIMIT_PER_CLASS = None  # Set to a number to limit images per class
SEGMENTATION_THRESHOLD = 0  # Pixels ABOVE this will be set to zero (inverted)
SEGMENTATION_THRESHOLD_UPPER = 140 # Pixels ABOVE this will be set to zero (inverted)

# Training parameters
EPOCHS = 12
BATCH_SIZE = 32

# Output paths
OUT_DIR = "./artifacts_grey"
OUT_NPZ = os.path.join(OUT_DIR, "rbc_data_grey.npz")

# Create output directories
for d in [OUT_DIR, f"{OUT_DIR}/models", f"{OUT_DIR}/plots", f"{OUT_DIR}/reports"]:
    os.makedirs(d, exist_ok=True)

print(f"Dataset directory: {os.path.abspath(DATASET_DIR)}")
print(f"Output directory: {os.path.abspath(OUT_DIR)}")
print(f"Original image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Crop factor: {CROP_FACTOR}")
print(f"Final cropped size: {CROPPED_SIZE}x{CROPPED_SIZE}")

## Data Preparation Functions

In [20]:
def get_class_names(dataset_dir: str) -> List[str]:
    """Get sorted list of class names from dataset directory."""
    return sorted([d.name for d in Path(dataset_dir).iterdir() if d.is_dir()])

def center_crop(im: Image.Image, crop_factor: float = CROP_FACTOR) -> Image.Image:
    """Apply center crop based on crop_factor (0 < crop_factor <= 1)."""
    if crop_factor <= 0 or crop_factor > 1:
        raise ValueError("crop_factor must be between 0 and 1")
    
    width, height = im.size
    new_width = int(width * crop_factor)
    new_height = int(height * crop_factor)
    
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    right = left + new_width
    bottom = top + new_height
    
    return im.crop((left, top, right, bottom))

def segment_image_inverted(im: Image.Image, threshold: float = SEGMENTATION_THRESHOLD, threshold_upper: float = SEGMENTATION_THRESHOLD_UPPER) -> Image.Image:
    """Apply INVERTED segmentation using PIL by setting pixel values ABOVE threshold to zero."""
    im_gray = im.convert('L')
    arr = np.asarray(im_gray, dtype=np.float32)
    
    # Apply inverted segmentation: pixels ABOVE threshold -> 0
    arr[arr <= threshold] = 0.0
    arr[arr >= threshold_upper] = 0.0
    return Image.fromarray(arr.astype(np.uint8), mode='L')

def process_image(im: Image.Image) -> np.ndarray:
    """Apply image processing steps to a PIL Image - GRAYSCALE VERSION with INVERSE SEGMENTATION FIRST and Sobel filter channel."""
    # Step 1: Segmentation
    im = segment_image_inverted(im, threshold=SEGMENTATION_THRESHOLD, threshold_upper=SEGMENTATION_THRESHOLD_UPPER)
    # Step 2: Center crop
    im = center_crop(im, crop_factor=CROP_FACTOR)
    # Step 3: Resize
    im = im.resize((CROPPED_SIZE, CROPPED_SIZE))
    # Step 4: (Optional median filter - currently commented out)
    #im = im.filter(ImageFilter.MedianFilter(size=3))
    
    # Convert to numpy array and normalize for grayscale channel
    arr_gray = np.asarray(im, dtype=np.float32) / 255.0
    
    # Apply Sobel filter after step 4
    # Sobel X and Y gradients
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32)
    
    from scipy.ndimage import convolve
    grad_x = convolve(arr_gray, sobel_x)
    grad_y = convolve(arr_gray, sobel_y)
    
    # Compute gradient magnitude
    sobel_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    # Normalize Sobel magnitude to [0, 1]
    sobel_magnitude = sobel_magnitude / (sobel_magnitude.max() + 1e-8)
    
    # Stack grayscale and Sobel channels
    arr_combined = np.stack([arr_gray, sobel_magnitude], axis=-1)
    
    return arr_combined

def load_images(dataset_dir: str, class_names: List[str]):
    """Load all JPG images from dataset directory."""
    X_list, y_list = [], []
    dataset_path = Path(dataset_dir)
    
    for cls_idx, cls_name in enumerate(class_names, start=1):
        # Use glob to get all jpg files
        jpg_files = list((dataset_path / cls_name).glob('*.jpg'))
        
        print(f"Loading {len(jpg_files)} images from {cls_name}...")
        
        for img_path in jpg_files:
            with Image.open(img_path) as im:
                arr = process_image(im)
            X_list.append(arr)
            y_list.append(cls_idx)
    
    return np.stack(X_list, axis=0), np.array(y_list, dtype=np.int64)

def split_data(X, y):
    """Split data into train, validation, and test sets."""
    # First split: train vs (val + test)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, 
        test_size=(VAL_FRAC + TEST_FRAC), 
        stratify=y, 
        random_state=42
    )
    
    # Second split: val vs test
    val_ratio = VAL_FRAC / (VAL_FRAC + TEST_FRAC)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, 
        test_size=(1 - val_ratio), 
        stratify=y_temp, 
        random_state=42
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test

print("Data preparation functions loaded!")

## Data Augmentation (Optional)

In [21]:
# Get class names
class_names = get_class_names(DATASET_DIR)
print(f"Classes found: {class_names}")
num_classes = len(class_names)

# Load images first using the processing pipeline
print("\nLoading images for augmentation...")
X_orig, y_orig = load_images(DATASET_DIR, class_names)
print(f"Loaded: X shape = {X_orig.shape}, y shape = {y_orig.shape}")

def augment_image(img_2channel: np.ndarray) -> np.ndarray:
    """Apply random translation and rotation to a 2-channel image."""
    from scipy.ndimage import rotate, shift
    
    # Random translation offset (-10% to +10% of image size)
    translate_x = (np.random.random() - 0.5) * 0.2 * CROPPED_SIZE
    translate_y = (np.random.random() - 0.5) * 0.2 * CROPPED_SIZE
    
    # Random rotation angle (-30 to +30 degrees)
    angle = (np.random.random() - 0.5) * 60
    
    # Apply transformations to both channels
    augmented = np.zeros_like(img_2channel)
    for ch in range(img_2channel.shape[-1]):
        # Shift
        shifted = shift(img_2channel[:, :, ch], [translate_y, translate_x], mode='nearest')
        # Rotate
        augmented[:, :, ch] = rotate(shifted, angle, reshape=False, mode='nearest')
    
    return augmented

# Perform data augmentation, generating 1 augmented copy of each image
print("\nApplying data augmentation...")
X_augmented = np.zeros_like(X_orig)
y_augmented = np.copy(y_orig)

for i in range(len(X_orig)):
    X_augmented[i] = augment_image(X_orig[i])

# Combine original and augmented data
X = np.concatenate((X_orig, X_augmented), axis=0)
y = np.concatenate((y_orig, y_augmented), axis=0)

print(f"After augmentation: X shape = {X.shape}, y shape = {y.shape}")

# Split data
print("\nSplitting data...")
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)
print(f"Train: {X_train.shape}")
print(f"Val:   {X_val.shape}")
print(f"Test:  {X_test.shape}")

# Save combined original + augmented data to the same OUT_NPZ file
meta = {
    "class_names": class_names,
    "original_img_size": IMG_SIZE,
    "crop_factor": CROP_FACTOR,
    "final_size": CROPPED_SIZE,
    "grayscale": True,
    "channels": 2,
    "augmented": True
}
np.savez_compressed(
    OUT_NPZ,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test,
    meta=json.dumps(meta)
)
print(f"\nSaved augmented data (original + augmented) to: {os.path.abspath(OUT_NPZ)}")

## Load Dataset

## Image Processing Output 


In [36]:
def visualize_processing_pipeline(dataset_dir, class_names, num_samples=3):
    """Visualize the image processing pipeline: Original RGB → Inverted Segmentation → Center Crop → Resize → Sobel Filter."""
    
    fig, axes = plt.subplots(num_samples, 5, figsize=(20, 4*num_samples))
    dataset_path = Path(dataset_dir)
    
    sample_count = 0
    for cls_name in class_names:
        if sample_count >= num_samples:
            break
        
        # Use glob to get first jpg file
        jpg_files = list((dataset_path / cls_name).glob('*.jpg'))
        if not jpg_files:
            continue
        
        img_path = jpg_files[0]
        
        # Step 1: Original RGB
        im_rgb = Image.open(img_path).convert("RGB")
        arr_rgb = np.asarray(im_rgb, dtype=np.float32) / 255.0
        
        # Step 2: Apply inverted segmentation FIRST
        im_segmented = segment_image_inverted(im_rgb, threshold=SEGMENTATION_THRESHOLD, threshold_upper=SEGMENTATION_THRESHOLD_UPPER)
        im_segmented_display = im_segmented.resize((CROPPED_SIZE, CROPPED_SIZE))
        arr_segmented = np.asarray(im_segmented_display, dtype=np.float32) / 255.0
        
        # Step 3: Center Crop (on segmented image)
        im_cropped = center_crop(im_segmented, crop_factor=CROP_FACTOR)
        im_cropped_display = im_cropped.resize((CROPPED_SIZE, CROPPED_SIZE))
        arr_cropped = np.asarray(im_cropped_display, dtype=np.float32) / 255.0
        
        # Step 4: Final processed version (this is what the model sees) - returns 2 channels
        arr_processed = process_image(im_rgb)  # Shape: (CROPPED_SIZE, CROPPED_SIZE, 2)
        arr_final = arr_processed[:, :, 0]  # Grayscale channel
        arr_sobel = arr_processed[:, :, 1]  # Sobel channel
        
        # Plot
        axes[sample_count, 0].imshow(arr_rgb)
        axes[sample_count, 0].set_title(f"{cls_name}\nOriginal RGB")
        axes[sample_count, 0].axis('off')
        
        axes[sample_count, 1].imshow(arr_segmented, cmap='gray')
        axes[sample_count, 1].set_title(f"Inverted Segmentation + Downsampling + Grayscale\n(threshold=[{SEGMENTATION_THRESHOLD} {SEGMENTATION_THRESHOLD_UPPER}])")
        axes[sample_count, 1].axis('off')
        
        axes[sample_count, 2].imshow(arr_cropped, cmap='gray')
        axes[sample_count, 2].set_title(f"Center Crop\n(factor={CROP_FACTOR})")
        axes[sample_count, 2].axis('off')
        
        axes[sample_count, 3].imshow(arr_final, cmap='gray')
        axes[sample_count, 3].set_title(f"Final Grayscale Channel\n(Model Input)")
        axes[sample_count, 3].axis('off')
        
        axes[sample_count, 4].imshow(arr_sobel, cmap='gray')
        axes[sample_count, 4].set_title(f"Final Sobel Edge Channel\n(Model Input)")
        axes[sample_count, 4].axis('off')
        
        im_rgb.close()
        sample_count += 1
    
    plt.suptitle(f'Image Processing Pipeline (Resize → Segmentation → Crop → Sobel) - Final: {CROPPED_SIZE}x{CROPPED_SIZE}x2', fontsize=16, y=1.00)
    plt.tight_layout()
    plt.savefig(f"{OUT_DIR}/plots/processing_pipeline.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f" Processing pipeline visualization saved to: {OUT_DIR}/plots/processing_pipeline.png")

# Visualize processing for 3 sample images
visualize_processing_pipeline(DATASET_DIR, class_names, num_samples=min(3, num_classes))

In [23]:
# Display count of images per class in the dataset
print("="*60)
print("IMAGE COUNT PER CLASS")
print("="*60)

dataset_path = Path(DATASET_DIR)
class_counts = {}

for cls_name in class_names:
    class_folder = dataset_path / cls_name
    # Count jpg files in each class folder
    jpg_files = list(class_folder.glob('*.jpg'))
    class_counts[cls_name] = len(jpg_files)

# Display as table
print(f"{'Class Name':<20} {'Image Count':>15}")
print("-"*60)
total_images = 0
for cls_name, count in class_counts.items():
    print(f"{cls_name:<20} {count:>15}")
    total_images += count

print("-"*60)
print(f"{'TOTAL':<20} {total_images:>15}")
print("="*60)

# Optional: Display as bar chart
plt.figure(figsize=(10, 5))
plt.bar(class_counts.keys(), class_counts.values(), color='steelblue', alpha=0.8, edgecolor='black')
plt.xlabel('Class Name')
plt.ylabel('Number of Images')
plt.title('Image Distribution Across Classes')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/plots/class_distribution.png", dpi=150, bbox_inches='tight')
plt.show()
print(f"\nClass distribution plot saved to: {OUT_DIR}/plots/class_distribution.png")

In [24]:
# Display sample images from each class in a 2x4 grid
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

dataset_path = Path(DATASET_DIR)

for idx, cls_name in enumerate(class_names):
    if idx >= 8:  # Only show 8 classes
        break
    
    # Get first image from this class
    class_folder = dataset_path / cls_name
    jpg_files = list(class_folder.glob('*.jpg'))
    
    if jpg_files:
        # Load and display the image
        img = Image.open(jpg_files[0])
        img_array = np.array(img)
        
        # Display the image
        if len(img_array.shape) == 3:  # Color image
            axes[idx].imshow(img_array)
        else:  # Grayscale
            axes[idx].imshow(img_array, cmap='gray')
        
        # Set title with class name and dimensions
        height, width = img_array.shape[:2]
        axes[idx].set_title(f"{cls_name}\n{width} x {height}", fontsize=12)
        axes[idx].axis('off')
    else:
        axes[idx].text(0.5, 0.5, 'No image found', ha='center', va='center')
        axes[idx].set_title(cls_name, fontsize=12)
        axes[idx].axis('off')

# Hide any unused subplots if less than 8 classes
for idx in range(len(class_names), 8):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig(f"{OUT_DIR}/plots/sample_images_per_class.png", dpi=150, bbox_inches='tight')
plt.show()
print(f"\nSample images grid saved to: {OUT_DIR}/plots/sample_images_per_class.png")

## ML Model Functions

In [25]:
def build_cnn(input_shape, num_classes,dropout_rate=0.5):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    # Block 1
    x = layers.Conv2D(16, 3, padding="same")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    #x = layers.Dropout(dropout_rate)(x)

    # Block 2
    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    #x = layers.Dropout(dropout_rate)(x)

    # Block 2
    x = layers.Conv2D(16, 3, padding="same")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    #x = layers.Dropout(dropout_rate)(x)

    # Block 3
    x = layers.Flatten()(x)
    x = layers.Dense(96, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(96, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs, name="tiny_cnn_regularized")
   


def build_big_cnn(input_shape, num_classes, dropout_rate=0.3):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    # Block 1
    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    x = layers.Dropout(0.25)(x)

    # Block 2
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    x = layers.Dropout(0.25)(x)

    # Block 3
    x = layers.Conv2D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    x = layers.Dropout(0.25)(x)

    # Dense layers
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return tf.keras.Model(inputs, outputs, name="improved_cnn")

def build_logreg_baseline(input_shape, num_classes):
    """Build a logistic regression baseline."""
    return tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Flatten(),
        layers.Dense(num_classes, activation="sigmoid")
    ])

print("Model building functions loaded!")

## Training and Evaluation Functions

In [26]:
def train_model(model, X_train, y_train, X_val, y_val, model_name, epochs=EPOCHS, batch_size=BATCH_SIZE):
    """Train a model and plot training curves."""
    # Convert labels to categorical
    y_train_cat = to_categorical(y_train - 1, num_classes=num_classes)
    y_val_cat = to_categorical(y_val - 1, num_classes=num_classes)
    
    # Compile model
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    # Train model
    print(f"\nTraining {model_name}...")
    
    history = model.fit(
        X_train, y_train_cat,
        validation_data=(X_val, y_val_cat),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    
    # Plot accuracy
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train")
    plt.plot(history.history["val_accuracy"], label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"{model_name} - Accuracy")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train")
    plt.plot(history.history["val_loss"], label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"{model_name} - Loss")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{OUT_DIR}/plots/{model_name}_training.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    return history

def evaluate_model(model, X_test, y_test, model_name):
    """Evaluate model and generate confusion matrix and classification report."""
    # Get predictions
    y_true = (y_test - 1).astype(int)
    y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
    
    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"\n{model_name} Test Accuracy: {acc:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))
    
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation="nearest", cmap="Blues")
    plt.title(f"{model_name} - Confusion Matrix")
    plt.colorbar()
    
    ticks = np.arange(num_classes)
    plt.xticks(ticks, class_names, rotation=45, ha="right")
    plt.yticks(ticks, class_names)
    
    # Add text annotations
    for i in range(num_classes):
        for j in range(num_classes):
            plt.text(j, i, str(cm[i, j]), ha="center", va="center",
                    color="white" if cm[i, j] > cm.max() / 2 else "black")
    
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(f"{OUT_DIR}/plots/{model_name}_cm.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    # Classification report
    report = classification_report(y_true, y_pred, target_names=class_names, digits=4)
    print(f"\n{model_name} Classification Report:")
    print(report)
    
    # Save report to file
    with open(f"{OUT_DIR}/reports/{model_name}_report.txt", "w") as f:
        f.write(f"Test Accuracy: {acc:.4f}\n\n")
        f.write(report)
    
    return acc

print("Training and evaluation functions loaded!")

## Training

In [27]:
input_shape = (CROPPED_SIZE, CROPPED_SIZE, 2)
dropout_rate = 0.3
cnn_model = build_cnn(input_shape, num_classes,dropout_rate)
cnn_model.summary()

# Train CNN
cnn_history = train_model(cnn_model, X_train, y_train, X_val, y_val, "tiny_cnn")

cnn_model.save(f"{OUT_DIR}/models/tiny_cnn.keras")
print(f"\nCNN model saved to: {OUT_DIR}/models/tiny_cnn.keras")

## Evaluate CNN Model

In [ ]:
cnn_accuracy = evaluate_model(cnn_model, X_test, y_test, "tiny_cnn")
from tensorflow.keras.utils import plot_model

plot_model(cnn_model, 
               to_file=f"{OUT_DIR}/plots/cnn_model_architecture.png", 
               show_shapes=True, 
               show_layer_names=True,
               rankdir='TB',  # Top to Bottom
               expand_nested=True, 
               dpi=150)

## Train Logistic Regression Baseline

In [ ]:
# Build logistic regression model
logreg_model = build_logreg_baseline(input_shape, num_classes)
logreg_model.summary()

# Train logistic regression
logreg_history = train_model(logreg_model, X_train, y_train, X_val, y_val, "logreg")

# Save model
logreg_model.save(f"{OUT_DIR}/models/logreg.keras")
print(f"\nLogistic regression model saved to: {OUT_DIR}/models/logreg.keras")

## Evaluate Logistic Regression Baseline

In [ ]:
logreg_accuracy = evaluate_model(logreg_model, X_test, y_test, "logreg")

## Compare Models

In [ ]:
# Compare model accuracies
print("\n" + "="*50)
print("MODEL COMPARISON")
print("="*50)
print(f"Tiny CNN Test Accuracy:        {cnn_accuracy:.4f}")
print(f"Logistic Regression Accuracy:  {logreg_accuracy:.4f}")
print(f"Improvement:                   {(cnn_accuracy - logreg_accuracy):.4f}")
print("="*50)

# Bar plot comparison
plt.figure(figsize=(8, 5))
models = ['Logistic Regression', 'Tiny CNN']
accuracies = [logreg_accuracy, cnn_accuracy]
colors = ['#FF6B6B', '#4ECDC4']

bars = plt.bar(models, accuracies, color=colors, alpha=0.8, edgecolor='black')
plt.ylabel('Test Accuracy')
plt.title('Model Comparison (Grayscale + Inverted Segmentation)')
plt.ylim(0, 1.0)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(f"{OUT_DIR}/plots/model_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
import tensorflow as tf
import visualkeras

from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model

def build_cnn(input_shape, num_classes,dropout_rate=0.5):

    inputs = layers.Input(shape=input_shape)
    x = inputs

    # Block 1
    x = layers.Conv2D(16, 3, padding="same")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    #x = layers.Dropout(dropout_rate)(x)

    # Block 2
    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    #x = layers.Dropout(dropout_rate)(x)

    # Block 2
    x = layers.Conv2D(16, 3, padding="same")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    #x = layers.Dropout(dropout_rate)(x)

    # Block 3
    x = layers.Flatten()(x)
    x = layers.Dense(96, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(96, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs, name="tiny_cnn_regularized")

input_shape = (CROPPED_SIZE, CROPPED_SIZE, 2)
dropout_rate = 0.5
cnn_model = build_cnn(input_shape, num_classes,dropout_rate)



# Text summary
cnn_model.summary()

# Diagram image (needs pydot + graphviz)
#plot_model(cnn_model, to_file="model_structure.png", show_shapes=True, expand_nested=True, dpi=160)
visualkeras.layered_view(cnn_model, legend=True)